In [57]:
from __future__ import print_function
import sys
import re
import numpy as np

from numpy import dot
from numpy.linalg import norm
from pyspark import SparkContext
from operator import add
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix


Dimention=20000
smallTrainFile='SmallTrainingData.txt'
TestFile='TestingData.txt'

In [58]:
def freqArray (listOfIndices, numberofwords):
    returnVal = np.zeros (Dimention)
    for index in listOfIndices:
        returnVal[index] = returnVal[index] + 1
    returnVal = np.divide(returnVal, numberofwords)
    return returnVal

def get_label(x):
    if 'AU' in x[0]:
        return 1
    else:
        return 0
def get_vector_regularization(x):
     return (np.multiply(x[0],x[1]),np.multiply(x[1],(np.e**float(x[2])/(1+np.e**float(x[2])))),
             x[0]*x[2],np.log(np.e**float(x[2])+1))
    
def get_y_pred(x):
    if x[1]>0:
        return 1
    else:
        return 0

In [62]:
sc.stop()
sc = SparkContext(appName="LogisticRegressionTask3")

d_corpus = sc.textFile(smallTrainFile)

numberOfDocs =d_corpus.count()

d_keyAndText = d_corpus.map(lambda x : (x[x.index('id="') + 4 : x.index('" url=')], x[x.index('">') + 2:][:-6]))
regex = re.compile('[^a-zA-Z]')
d_keyAndListOfWords = d_keyAndText.map(lambda x : (str(x[0]), regex.sub(' ', x[1]).lower().split()))

allWords = d_keyAndListOfWords.flatMap(lambda x: ((i,1) for i in x[1]))

allCounts = allWords.reduceByKey(lambda x,y: x+y)

topWords = allCounts.top(Dimention, lambda x : x[1])

topWordsK = sc.parallelize(range(Dimention))
dictionary = topWordsK.map (lambda x : (topWords[x][0], x))

allWordsWithDocID = d_keyAndListOfWords.flatMap(lambda x: ((j, x[0]) for j in x[1]))

allDictionaryWords = dictionary.join (allWordsWithDocID)


justDocAndPos = allDictionaryWords.map (lambda x: (x[1][1], x[1][0]))

allDictionaryWordsInEachDoc = justDocAndPos.groupByKey()

allDocsAsNumpyArrays = allDictionaryWordsInEachDoc.map(lambda x: (x[0], freqArray(x[1],len(x[1]))))
zeroOrOne = allDocsAsNumpyArrays.map(lambda x: (x[0], np.clip (np.multiply (x[1], 9e9), 0, 1)))
dfArray = zeroOrOne.reduce(lambda x1, x2: ("", np.add(x1[1], x2[1])))[1]
multiplier = np.full(Dimention, numberOfDocs)
idfArray = np.log(np.divide(np.full(Dimention, numberOfDocs), dfArray))

allDocsAsNumpyArraysTFidf = allDocsAsNumpyArrays.map(lambda x: (x[0], np.multiply(x[1], idfArray)))


test_corpus = sc.textFile(TestFile)

numberOfDocs =test_corpus.count()

test_keyAndText = test_corpus.map(lambda x : (x[x.index('id="') + 4 : x.index('" url=')], x[x.index('">') + 2:][:-6]))
regex = re.compile('[^a-zA-Z]')
test_keyAndListOfWords = test_keyAndText.map(lambda x : (str(x[0]), regex.sub(' ', x[1]).lower().split()))

t_allWordsWithDocID = test_keyAndListOfWords.flatMap(lambda x: ((j, x[0]) for j in x[1]))

t_allDictionaryWords = dictionary.join (t_allWordsWithDocID)

t_justDocAndPos = t_allDictionaryWords.map (lambda x: (x[1][1], x[1][0]))

t_allDictionaryWordsInEachDoc = t_justDocAndPos.groupByKey()


t_allDocsAsNumpyArrays = t_allDictionaryWordsInEachDoc.map(lambda x: (x[0], freqArray(x[1],len(x[1]))))

t_zeroOrOne = t_allDocsAsNumpyArrays.map(lambda x: (x[0], np.clip (np.multiply (x[1], 9e9), 0, 1)))
t_dfArray = t_zeroOrOne.reduce(lambda x1, x2: ("", np.add(x1[1], x2[1])))[1]
t_idfArray = np.log(np.divide(np.full(20000, numberOfDocs), 1+t_dfArray))
t_allDocsAsNumpyArraysTFidf = t_allDocsAsNumpyArrays.map(lambda x: (x[0], np.multiply(x[1], t_idfArray)))

# With Regularization
R_current2 = np.random.normal(0, 0.1, Dimention)
learningRate = 0.1
num_iteration = 9999999
oldCost = 0

# I have tested many lambda, lambda=0.0001 can get a good result
lamda = 0.00001

allDocsAsNumpyArraysTFidf.cache()

for i in range(num_iteration):

    Y_X_Theta = allDocsAsNumpyArraysTFidf.map(lambda x: ((get_label(x), x[1], np.dot(x[1], R_current2))))
    Gradient = Y_X_Theta.map(get_vector_regularization)
    GradientSum = Gradient.reduce(lambda x, y: ((x[0] + y[0]), (x[1] + y[1]), (x[2] + y[2]), (x[3] + y[3])))

    # calculate cost:
    cost = GradientSum[3] - GradientSum[2] + lamda * (np.square(R_current2).sum())

    oldp = np.sqrt(np.square(R_current2).sum())

    R_current2 = R_current2 - learningRate *( (-GradientSum[0] + GradientSum[1])+2*lamda*R_current2)

    p = np.sqrt(np.square(R_current2).sum())

    # Stop if the cost is not descreasing
    if abs(oldp - p) <= 0.008:
        break

    if (cost <= oldCost):
        learningRate = learningRate * 1.05
        oldCost = cost

    if (cost > oldCost):
        learningRate = learningRate * 0.5
        oldCost = cost

    print("Iteration No.=", i, " Cost=", cost)
    
y_ypred2=t_allDocsAsNumpyArraysTFidf.map(lambda x: (x[0],1 if 'AU' in x[0] else 0,0 if np.dot(x[1], R_current2) <0 else 1)).collect()
  
y2=[]
ypred2=[]
counter=0
for i in y_ypred2:
    y2.append(i[1])
    ypred2.append(i[2])
    if i[1] == 0 and i[2]==1:
        docIDList = t_allDocsAsNumpyArraysTFidf.filter(lambda x: x[0] == i[0]).collect()
        print(docIDList[0][0])
        counter+=1
    if counter >=3:
        break
        
print('f1 score:', f1_score(y2, ypred2, average='binary'))
print('confusion matrix:', confusion_matrix(y2, ypred2))



Iteration No.= 0  Cost= 2385.856111198769
Iteration No.= 1  Cost= 2286.2188405867983
Iteration No.= 2  Cost= 2239.1706535905487
Iteration No.= 3  Cost= 2191.3567268133243
Iteration No.= 4  Cost= 2142.8383005955275
Iteration No.= 5  Cost= 2093.682272400763
Iteration No.= 6  Cost= 2043.961044251932
Iteration No.= 7  Cost= 1993.7522690117019
Iteration No.= 8  Cost= 1943.138496091151
Iteration No.= 9  Cost= 1892.2067190948203
Iteration No.= 10  Cost= 1841.0478294923196
Iteration No.= 11  Cost= 1789.755982123528
Iteration No.= 12  Cost= 1738.4278803702944
Iteration No.= 13  Cost= 1687.1619911226956
Iteration No.= 14  Cost= 1636.0577020766866
Iteration No.= 15  Cost= 1585.2144362212289
Iteration No.= 16  Cost= 1534.730740403616
Iteration No.= 17  Cost= 1484.7033664213354
Iteration No.= 18  Cost= 1435.2263640330932
Iteration No.= 19  Cost= 1386.390205511365
Iteration No.= 20  Cost= 1338.280960824359
Iteration No.= 21  Cost= 1290.9795412388924
Iteration No.= 22  Cost= 1244.5610271303622
Iterat

Iteration No.= 188  Cost= 4.355873648425545
Iteration No.= 189  Cost= 4.348474665745513
Iteration No.= 190  Cost= 4.341967413730185
Iteration No.= 191  Cost= 4.336261934917713
Iteration No.= 192  Cost= 4.331274018191985
Iteration No.= 193  Cost= 4.326925168340848
Iteration No.= 194  Cost= 4.323142566057281
Iteration No.= 195  Cost= 4.319859013442315
Iteration No.= 196  Cost= 4.317012860193676
Iteration No.= 197  Cost= 4.314547905984096
Iteration No.= 198  Cost= 4.312413275079654
Iteration No.= 199  Cost= 4.310563260031745
Iteration No.= 200  Cost= 4.308957132320092
Iteration No.= 201  Cost= 4.307558919078871
Iteration No.= 202  Cost= 4.306337146494823
Iteration No.= 203  Cost= 4.305264552037253
Iteration No.= 204  Cost= 4.304317769299839
Iteration No.= 205  Cost= 4.303476990761055
Iteration No.= 206  Cost= 4.302725615118364
Iteration No.= 207  Cost= 4.302049886866845
Iteration No.= 208  Cost= 4.30143853638536
Iteration No.= 209  Cost= 4.300882428867716
Iteration No.= 210  Cost= 4.30037